# Introduction

Derivatives are used to solve a large variety of modern-day problems. There are three general methods used to calculate derivatives:
1. Symbolic differentiation 
2. Numerical differentiation
3. Automatic differentiation 

Symbolic differentiation can be very useful, but there are some functions that do not have a symbolic derivative. Additionally, symbolic differentiation can be very costly, as it may recalculate the same expressions many times, or the expression for the derivative may grow exponentially. Sometimes we can avoid these issues by numerically differentiating our function. Often this means using finite differences. The method of finite differences calculates derivative at point $x$ by using the following definition:

$$f'(x) = \lim_{h\to 0} f(x) \frac{f(x+h)-f(x)}{h}$$ 

Finite differences can also be very effective in certain situations. However, as with symbolic differentiation, finite differences has its problems. The biggest issue is that to obtain the most accurate estimate of $f'(x)$, we would like to make $h$ as small as possible; in fact, we would like $h$ to be infinitely small. However, we cannot *actually* make $h$ zero, and thus we must compromise and choose some small-but-not-zero value for $h$, which brings us to our second problem: we cannot precisely represent all numbers. Our machines only have a certain level of precision. When we compute our derivatives numerically we introduce error by approximating values to their closest machine equivalent. To avoid these issues, we turn to our third approach: automatic differentiation. 

# Background

Automatic differentiation (AD) allows us to calculate the derivative to machine precision while avoiding symbolic differentiation's shortcomings. Our package implements on version of AD, the forward mode, by using an extension of the real numbers called the "dual numbers." The forward mode of AD finds the derivative of all intermediate variables with respect to our independent variable and combines them into a final derivative using the chain rule.

AD can also be used in "reverse mode," which we will not discuss in detail her, but this method shares many of the same characteristics as forward mode. However, the reverse mode computes derivatives of the dependent variable with respect to the intermediate variables. 

#### Dual Numbers
To carry out the forward mode AD we utilize dual numbers. Dual numbers are defined as numbers of the form $x + x'\epsilon$, where $\epsilon^2=0$ and $x \in \mathbb{R}^n$. We use operator overloading to redefine elementary operations to suit our problem. To see why this is useful, let's examine how dual numbers behave under different mathematical operations:

Addition: $(x+x'\epsilon) + (y + y'\epsilon) = x+y + (x'+y')\epsilon$

Subtraction: $(x+x'\epsilon) - (y + y'\epsilon) = x-y + (x'-y')\epsilon$

So far, this is as we might expect.

Multiplication: $(x+x'\epsilon) \times (y + y'\epsilon) = xy + y(x')\epsilon+ x(y')\epsilon$

Our definition of $\epsilon$ allows the multiplication of dual numbers to behave like the product rule.

Division: $\frac{(x+x'\epsilon)}{(y + y'\epsilon)} = \frac{(x+x'\epsilon)(y - y'\epsilon)}{(y + y'\epsilon)(y - y'\epsilon)} = \frac{xy+xy'\epsilon-yx'\epsilon}{y^2} = \frac{x}{y}+\epsilon \frac{xy'-yx'}{y^2}$

Division also follows rules for derivatives.

Finally, observe how functions of dual numbers behave:

$f(x+x'\epsilon) = f(x)+\epsilon f'(x)x'$

Which implies the following:

$g(f(x+x'\epsilon)) = g(f(x)+\epsilon f'(x)x') = g(f(x))+\epsilon g'(f(x))f'(x)x'$

The above example illustrates how dual numbers can be used to simultaneously calculate the value of a function at a point, $g(f(x))$, and the derivative, $g'(f(x))f'(x)x'$.


#### Tracing the computational graph
By keeping track of the intermediate values of the derivative we can calculate the derivative of composition of many elementary functions. We can picture this decomposition as a graph or table. For example, consider the following function$^{1}$: $$f\left(x, y, z\right) = \dfrac{1}{xyz} + \sin\left(\dfrac{1}{x} + \dfrac{1}{y} + \dfrac{1}{z}\right).$$

If we want to evaluate $f$ at $\left(1, 2, 3\right)$, we can construct the following table which keeps track for the elementary function, current value, and the elementary function derivative (evaluated with respect to all our variables).

| Trace | Elementary Function | Current Value | Elementary Function Derivative | $\nabla_{x}$ Value  | $\nabla_{y}$ Value  | $\nabla_{z}$ Value  |
| :---: | :-----------------: | :-----------: | :----------------------------: | :-----------------: | :-----------------: | :-----------------: |
| $x_{1}$ | $x_{1}$ | $1$ | $\dot{x}_1$ | $1$ | $0$ | $0$ | 
| $x_{2}$ | $x_{2}$ | $2$ | $\dot{x}_2$ | $0$ | $1$ | $0$ | 
| $x_{3}$ | $x_{3}$ | $3$ | $\dot{x}_3$ | $0$ | $0$ | $1$ | 
| $x_{4}$ | $1/x_{1}$ | $1$ | $-\dot{x}_{1}/x_{1}^{2}$ | $-1$ | $0$ | $0$ | 
| $x_{5}$ | $1/x_{2}$ | $\frac{1}{2}$ | $-\dot{x}_{2}/x_{2}^{2}$ | $0$ | $-\frac{1}{4}$ | $0$ | 
| $x_{6}$ | $1/x_{3}$ | $\frac{1}{3}$ | $-\dot{x}_{3}/x_{3}^{2}$ | $0$ | $0$ | $-\frac{1}{9}$ | 
| $x_{7}$ | $x_4 x_5 x_6$ | $\frac{1}{6}$ | $x_4(x_5\dot{x}_6 + x_6\dot{x}_5) + x_5x_6\dot{x}_4$ | $-\frac{1}{6}$ | $-\frac{1}{12}$ | $-\frac{1}{18}$ | 
| $x_{8}$ | $x_4 + x_5 + x_6$ | $\frac{11}{6}$ | $\dot{x}_4 + \dot{x}_5 + \dot{x}_6$ | $-1$ | $-\frac{1}{4}$ | $-\frac{1}{9}$ | 
| $x_{9}$ | $sin(x_8)$ | $sin(\frac{11}{6})$ | $cos(x_8)\dot{x}_8$ | $-cos(\frac{11}{6})$ | $-\frac{1}{4}cos(\frac{11}{6})$ | $-\frac{1}{9}cos(\frac{11}{6})$ | 
| $x_{10}$ | $x_7 + x_9$ | $sin(\frac{11}{6})+\frac{1}{6}$ | $\dot{x}_7 + \dot{x}_9$ | $-cos(\frac{11}{6})-\frac{1}{6}$ | $-\frac{1}{4}cos(\frac{11}{6})-\frac{1}{12}$ | $-\frac{1}{9}cos(\frac{11}{6})-\frac{1}{18}$ | 

As this example shows, we can use AD for both scalar and vector functions. AD can also be used for vector valued functions. The follow sections will make the implementation of these varients clear.

$^1$Example from Harvard CS207 Homework 4

# Package Usage

## Installation

Please follow these two steps in sequence to install:

1. Clone https://github.com/autodiff-cs207/AutoDiff.git
2. After cloning, please run:

   ` python setup.py install`


## Package Import Design

`>>> from AutoDiff import DiffObj, Variable, Constant`
`>>> from AutoDiff import MathOps as mo`

`>>> from AutoDiff import MathOps as ops`

**We have created a comprehensive Jupyter Notebook, which demostrates the functionality of our AutoDiff package. The notebook may be found at [this link](https://github.com/autodiff-cs207/AutoDiff/blob/master/AutoDiff/AutoDiff_Demo.ipynb). We have also provided a demostration of how our AutoDiff may be used to calculate roots using Newton-Raphson method for the following function:**

$$
f(x) = 5^{\left(1 + sin\left(log\left(5 + x^2\right)\right)\right)} - 10
$$

## Example package use
Each variable and constant term which appears in a function that the user wants to differentiate, should be an instance of the classes `Variable` and `Constant` respectively. 

### Declaring Variables and Constants
All variables and constants need to be instances of the classes `Variable` and `Constant` respectively.
`>>> x = Variable('x') `
`>>> y = Variable('y') `
`>>> c1 = Constant('c1', 5.0) `

### Evaluating Function and Calculating Gradients
Suppose the user wants to differentiate $5sin(x + y)$ at $x= \pi/2$ and $y= \pi/3$:

1. First declare the variables and constants which will be used in the function:

    `>>> x = Variable('x') `
    
    `>>> y = Variable('y') `
    
    `>>> c1 = Constant('c1', 5.0) `

2. Next assign the desired function to $f$:

    `>>> f = c1*mo.sin(x + y)`
    
   $f$ is now an object of the class DiffObj.

3. Now create a dictionary which stores the values at which you want to evaluate $f$ and its gradient:

    `>>> val_dict = {'x': math.pi/2, 'y': math.pi/3} `

4. Now we are ready to evaluate our function. We can do this by invoking the method `get_val` on $f$.
    
    `>>> print(f.get_val(val_dict)`
    
    ` 2.5000000000000018`
   
   Simiarly we can now get the gradient by invoking `get_der` on $f$:
   
    `>>> print(f.get_der(val_dict)`
    
    ` {'x': -4.330127018922193, 'y': -4.330127018922193}`
    
5. Lastly, if the user just needs the gradient with respect to $x$, it can be done as follows:

    `>>> f.get_der(val_dict, with_respect_to=['x'])`
    
    ` {'x': -4.330127018922193}`

# Software Organization
### Directory Structure
```
AutoDiff/
    AutoDiff/
        AutoDiff.py
        tests/
            test_AutoDiff.py
        README.md
        AutoDiff_Demo.ipynb
        __init__.py
        setup.py
        LICENSE
```
### Modules and Functionality
We currently have a module named AutoDiff which contains the classes DiffObj(), Variable(DiffObj), Constant(DiffObj), MathOps(DiffObj). Their basic functionality is defined below in the Implementation Details section. Within these modules, we use the math library to access functions like math.sin().

### Testing
Our test suite resides in the AutoDiff directory (as shown in the directory structure above), and we have used both TravisCI and Coveralls to automate testing. In addition, we have also written DocTest code for each class function and our package passes all doctest by running doctest.testmod().

### Distribution
We eventually plan to dsitribute via PyPI, however for now we have provided package installation instructions above. There are two steps to it: (1) Clone our AutoDiff repo (we provide the repo path above), and (2) Run the `setup.py` file, which we provide with the repo.

# Implementation Details

Our basic approach is to capture the essence of Chain Rule through use of recursion for calculating derivatives. Each mathematical expression, from a simple constant (like 5.0) or variable (such as 'x'), to complex expressions like $\log(sin(x^2 + yz))$, are all instances of our class `DiffObj` or of sub-classes which inherit from `DiffObj`. And the `DiffObj` class requies that anything which inherits from it, implements two functions: `get_val` for evaluating a mathematical expression, and `get_der` for evaluating gradient of a mathematical expression. This allows us to use Chain Rule by recursively calling these two functions on parts of an expresison.

## Functionality
Our AutoDiff package is currently capable of calculating gradients for real valued functions of multiple variables. For functions with more than one variable, the package has the capability to return partial derivatives with respect to all variables that appear in the user's function expression.

We support the following elementary math operators currently:
* log (Natural log)
* sin (sine function from trigonometry)
* cos (cosine function from trigonometry)
* tan (tangent function from trigonometry)
* exp (exponentiation with base equal to Euler's Number $e$)

Further, we overload the following Python math operators:
* \_\_add\__ and \__radd\__ (allows addition of two DiffObj instances)
* \__sub\__ and \__rsub\__ (allows subtraction between two DiffObj instances)
* \__truediv\__ and \__rtruediv\__ (allows division between two DiffObj instances)
* \__mul\__ and \__rmul\__ (allows multiplication between two DiffObj instances)
* \__pow\__ and \__rpow\__ (allows exponentiation between two DiffObj instances)
* \__neg\__ (unary operator for negation of a DiffObj instance)

## Class Structure
We have implemented the following classes in our package:
1. Class DiffObj()
2. Class Variable(DiffObj)
3. Class Constant(DiffObj)
4. Class MathOps(DiffObj)

These are described along with their Class Attributes and Class Methods below.

### 1. Class DiffObj()
Any function for which a user wants to evaluate its value and gradient, will be represented by an instance of this class DiffObj, or by instances of classes which inherit from DiffObj (e.g. class Variable, class Constant etc.) A mathematical equivalent of a DiffObj object will be:
 * a constant such as $5.0$, which we have implemented via a Sub-class 'Constant'
 * a variable such as $x$, which we have implemented via a Sub-class 'Variable'
 * a mathematical expression such as $x^2 + sin(y)$.

DiffObj enforces that each class which inherits from it, must implement two functions:

    CLASS FUNCTIONS
    ==================
    The functions get_val and get_der are exposed to the user, that is, a user of our package can
    call these functions.

    (1) get_val:        This is used to evaluate the function represented by a DiffObj instance at
                        a particular point.

    (2) get_der:        This is used to evalate the gradient of the function repreesnted by a DiffObj
                        instance, at a particular point.
   
    CLASS ATTRIBUTES
    ================
    The attributes are not meant to be used by an end-user of our package, and they are meant for internal
    computation.

    name_list:          A list of strings, where each item in the list represents the variables inside
                        the function represented by this DiffObj. E.g. for f(x,y) = x + y, the name_list
                        for a DiffObj representing f will be ['x', 'y'] (assuming the x.name_list = ['x']
                        and y.name_list = ['y'].
    operator:           A single string representing the "operator". By default, DiffObj assumes that it
                        represents two DiffObj's connected by an binary operator such as 'add'. However,
                        we use the same definition for unary operators such as negation or cosine.
    operand_list:       A list of two DiffObjs, which together with self.operator, comprise this instance
                        of DiffObj.

    CLASS FUNCTIONS
    ================
    get_val(self, value_dict)
        INPUT
        ======
        value_dict:     A dictionary, whose keys are strings representing variables which feature
                        in the formula represented by this DiffObj. The values at those keys are
                        the values at which the formula representing this DiffObj will be evaluated.

                        E.g. For a DiffObj which represents the function f(x,y) = x + y, the value_dict
                        argument may look like value_dict = {'x': 10, 'y': 5}
        OUTPUT
        ======

        DOCTEST
        ======
        >>> z=x+y
        >>> z.get_val({'x':1,'y':1})
        2


        result:         A floating point number, which equals the evaluation of the function
                        represented by this DiffObj, at the variable values given by val_dict.

    get_der(self, value_dict, with_respect_to=None)
        INPUT
        ======
        value_dict:         A dictionary, whose keys are strings representing variables which feature
                            in the formula represented by this DiffObj. The values at those keys are
                            the values at which the gradient of formula representing this DiffObj will
                            be evaluated.

                            E.g. For a DiffObj which represents the function f(x,y) = x + y, the value_dict
                            argument may look like value_dict = {'x': 10, 'y': 5}
        with_respect_to:    A list of strings representing variables, with respect to which we want the
                            gradient of this DifObj. By default, if this list is not provided, then the
                            gradient with respect to all variables featuring in the DiffObj is returned.

        OUTPUT
        ======
        result:             A dictionary, whose keys are strings representing variables which feature
                            in the formula represented by this DiffObj. The value associated withe each
                            key is a floating point number which is the partial derivative of this DiffObj
                            with respect to that variable.

        DOCTEST
        ======
        >>> z=x+y
        >>> z.get_der({'x':0,'y':0})
        {'y': 1, 'x': 1}
        
    Other class functions: These include the overloaded operators listed in the functionality section above. 
                            We have provided detailed documentation for these overloaded functions inside our code. 

### 2. Class Variable(DiffObj)
This subclass inherits from DiffObj, and is basically used for representing a variable such as x or y.

    All variables inside a function whose derivative and value a user wants to calculate,
    will be instances of the Variable class, which inherits from DiffObj and implements
    get_val and get_der

    CLASS ATTRIBUTES
    ================
    var_name:           A string, which is unique to this Variable instance.
                        E.g. x = Variable('x')

    CLASS FUNCTIONS
    ===============
    This implements get_val and get_der, a description of which is provided in the
    Super-class DiffObj.

### 3. Class Constant(DiffObj)

    All constants inside a function whose derivative and value a user wants to calculate,
    will be instances of the Constant class, which inherits from DiffObj and implements
    get_val and get_der

    CLASS ATTRIBUTES
    ================
    const_name:         A string, which is unique to this Constant instance.
    const_val:          An int or float number, which will be the value assigned to this instance.

                        E.g. c = Constant('c', 10.0)

    CLASS FUNCTIONS
    ===============
    This implements get_val and get_der, a description of which is provided in the
    Super-class DiffObj. As expected, get_val simply returns self.const_val while
    get_der will return 0.

### 4. Class MathOps()
    This class inherits from the DiffObj class. It implements non-elementary unary functions
    including: sin, cos, tan, log, exp.

    INSTANTIATION
    ===============
    If a is of type DiffObj, then the invoking the constructor as follows will return an
    object b of type MathOps:

    b = MathOps.sin(a)

    CLASS ATTRIBUTES
    ================
    The attributes are not meant to be used by an end-user of our package, and they are meant for internal
    computation.

    name_list:          A list of strings, where each item in the list represents the variables inside
                        the function represented by this DiffObj. E.g. for f(x,y) = x + y, the name_list
                        for a DiffObj representing f will be ['x', 'y'] (assuming the x.name_list = ['x']
                        and y.name_list = ['y'].
    operator:           A string, such as 'sin' or 'log', which represents one of the unary math operators
                        implemented by this class.
    operand_list:       A list of length 1 containing the DiffObj which the user has passed as an argument
                        to one of the classmethods of MathOps.
    CLASS FUNCTIONS
    ================
    Note: This class implements classmethods named 'sin', 'cos', 'tan', 'log' and 'exp', and these
    classmethods basically return an instance of type DiffObj, which supports get_val and get_der
    for functions like sin, log, cos, tan, exp.

## Core Data Structures

There are two core data structures in our implementation:

1. **Lists**: The name_list (a list of strings) representing variable names, that is stored in every Diffobj instance to indicate the variables influencing that instance. Eg. for the DiffObj w, where w represents sin(x)+y, the name_list of Variable x is ['x'], the name_list of Variable y is ['y'] and the name_list of w is ['x','y'].

2. **Dictionaries**: The dictionary value_dict, an argument of DiffObj.get_der, containing names and values that indicate the point in space at which we need to compute derivative and evaluate an expression, for example in w.get_val(value_dict). We also use Dictionarie for storing partial derivatives with respect to variables.

## External Dependencies

As of now we believe we will use the following third party libraries:
1. `Math`

We may use `NumPy` when we extend our AD package.

## Dealing with Elementary Functions
In our design, we have provided a Class called MathOps, which allows calculations related to elementary functions such as $sin$ and $log$. In line with our philosophy to make anything which is differentiable, an instance of `DiffObj`, out `Mathops` class provides certain classmethods, whose job is to wrap an expression of the form $sin(some\_diff\_obj)$ into another `DiffObj`, which supports the `get_val` and `get_der` functions which are crucial to our implementation of the Chain Rule.

# Future Extensions
We have planned the following enhancements to our AD package:

1.	Future AD package should support Vector variables. Vector variables can easily be implemented as a list of Variables, but such implementation may be inefficient. The challenge for us is to use a better structure to represent Vectors and algorithms to calculate derivatives with respect to Vectors, without too much Python structure overhead.
2.	Further support on matrices. We are aiming to implement Backpropogation. We will explore how to incorporate matrix operations into our code.
3.	We aim to support a more natural coding experience. For example, as of now, the only way to use a constant in a function is to make the constant an instance of the Constant class. We are cognizant of the fact that the Constnat class is not really providing much functionality. We will aim to allow users to directly use constant numbers in their mathetmatical expressions.